# **Homework 1: COVID-19 Cases Prediction (Regression)**

Objectives:
* Solve a regression problem with deep neural networks (DNN).
* Understand basic DNN training tips.
* Familiarize yourself with PyTorch.

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2022-spring@googlegroups.com

# Download data
If the Google Drive links below do not work, you can download data from [Kaggle](https://www.kaggle.com/t/a3ebd5b5542f0f55e828d4f00de8e59a), and upload data manually to the workspace.

In [6]:
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

zsh:1: command not found: gdown
zsh:1: command not found: gdown


# Import packages

In [27]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Some Utility Functions

You do not need to modify this part.

In [101]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())
    preds = torch.cat(preds, dim=0).numpy()
    return preds

# Dataset

In [102]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Neural Network Model
Try out different model architectures by modifying the class below.

In [103]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions.
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# Feature Selection
Choose features you deem useful by modifying the function below.

In [235]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

##RF



In [115]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

def select_feat(train_data, valid_data, test_data, threshold=0.0001, select_all=True):
    '''Selects features using Random Forest feature importance'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        # 创建一个RandomForestRegressor模型，可以调整超参数
        model = RandomForestRegressor(n_estimators=100, max_depth=10)  # 例如，设置树的数量和最大深度
        
        # 拟合模型到训练数据
        model.fit(raw_x_train, y_train)
        
        # 获取特征重要性
        feature_importances = model.feature_importances_
        
        # 创建特征选择器，基于特征重要性
        selector = SelectFromModel(model, threshold=threshold)
        
        # 拟合选择器到训练数据
        selector.fit(raw_x_train, y_train)
        
        # 获取选择的特征的索引
        feat_idx = selector.get_support(indices=True)
        

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


##Lasso

In [90]:
from sklearn.linear_model import Lasso

def select_feat(train_data, valid_data, test_data, alpha=0.001):
    '''Selects features using Lasso Regression'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    # 创建 Lasso 模型
    model = Lasso(alpha=alpha)

    # 拟合 Lasso 模型到训练数据
    model.fit(raw_x_train, y_train)

    # 获取特征的系数，系数不为零的特征将被选择
    coef = model.coef_
    
    # 获取非零系数的特征索引
    feat_idx = [i for i in range(len(coef)) if coef[i] != 0]
    

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


##correlation pearson

In [193]:
import numpy as np

def select_feat(train_data, valid_data, test_data, select_all=True, threshold=0.2):
    '''Selects useful features to perform regression based on Pearson correlation'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        # Calculate Pearson correlation coefficients between features and target variable
        corr_with_target = [abs(np.corrcoef(raw_x_train[:, i], y_train)[0, 1]) for i in range(raw_x_train.shape[1] - 1)]

        # Select features with correlation above the specified threshold
        feat_idx = [i for i, corr in enumerate(corr_with_target) if corr >= threshold]
    else:
        # TODO: Select suitable feature columns based on Pearson correlation.
        feat_idx = [0, 1, 2, 3, 4]  # Replace with your selection criteria.

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


In [200]:
import numpy as np
from scipy.stats import spearmanr

def select_feat(train_data, valid_data, test_data, select_all=True, threshold=0.2):
    '''Selects useful features to perform regression based on Spearman correlation'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        # Calculate Spearman correlation coefficients between features and target variable
        corr_with_target = [abs(spearmanr(raw_x_train[:, i], y_train).correlation) for i in range(raw_x_train.shape[1] - 1)]

        # Select features with correlation above the specified threshold
        feat_idx = [i for i, corr in enumerate(corr_with_target) if corr >= threshold]
    else:
        # TODO: Select suitable feature columns based on Spearman correlation.
        feat_idx = [0, 1, 2, 3, 4]  # Replace with your selection criteria.

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


In [207]:
import numpy as np
from scipy.stats import kendalltau

def select_feat(train_data, valid_data, test_data, select_all=True, threshold=0.2):
    '''Selects useful features to perform regression based on Kendall Tau correlation'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        # Calculate Kendall Tau correlation coefficients between features and target variable
        corr_with_target = [abs(kendalltau(raw_x_train[:, i], y_train).correlation) for i in range(raw_x_train.shape[1] - 1)]

        # Select features with correlation above the specified threshold
        feat_idx = [i for i, corr in enumerate(corr_with_target) if corr >= threshold]
    else:
        # TODO: Select suitable feature columns based on Kendall Tau correlation.
        feat_idx = [0, 1, 2, 3, 4]  # Replace with your selection criteria.

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


In [242]:
from sklearn.ensemble import RandomForestRegressor

def select_feat(train_data, valid_data, test_data, select_all=True, num_features=5):
    '''Selects useful features to perform regression based on feature importance from Random Forest'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        # Train a Random Forest model to calculate feature importances
        forest = RandomForestRegressor(n_estimators=500)
        forest.fit(raw_x_train, y_train)
        
        # Get feature importances and select the top features
        importances = forest.feature_importances_
        feat_idx = np.argsort(importances)[::-1][:num_features]
    else:
        # TODO: Select suitable feature columns based on feature importance from Random Forest.
        feat_idx = [0, 1, 2, 3, 4]  # Replace with your selection criteria.

    return raw_x_train[:, feat_idx], raw_x_valid[:, feat_idx], raw_x_test[:, feat_idx], y_train, y_valid


# Training Loop

In [243]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm.
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# Configurations
`config` contains hyper-parameters for training and the path to save your model.

In [244]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 3000,     # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [245]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days)
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (2160, 117)
valid_data size: (539, 117)
test_data size: (1078, 116)
number of features: 20


# Start training!

In [246]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/3000]: 100%|██████████| 9/9 [00:00<00:00, 402.86it/s, loss=126]


Epoch [1/3000]: Train loss: 182.6228, Valid loss: 132.6119
Saving model with loss 132.612...


Epoch [2/3000]: 100%|██████████| 9/9 [00:00<00:00, 649.72it/s, loss=66.8]


Epoch [2/3000]: Train loss: 102.6994, Valid loss: 88.3107
Saving model with loss 88.311...


Epoch [3/3000]: 100%|██████████| 9/9 [00:00<00:00, 393.27it/s, loss=39.3]


...

Epoch [2992/3000]: Train loss: 1.1457, Valid loss: 1.1780


Epoch [2993/3000]: 100%|██████████| 9/9 [00:00<00:00, 450.52it/s, loss=0.813]


Epoch [2993/3000]: Train loss: 1.1000, Valid loss: 1.2412


Epoch [2994/3000]: 100%|██████████| 9/9 [00:00<00:00, 91.01it/s, loss=0.951]


Epoch [2994/3000]: Train loss: 1.1026, Valid loss: 1.2178

Model is not improving, so we halt the training session.


# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information.

In [247]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6006 (pid 258), started 2:28:15 ago. (Use '!kill 258' to kill it.)

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [248]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')

100%|██████████| 5/5 [00:00<00:00, 1216.23it/s]


# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)